In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-playoff-seasonal-data/team_season.txt
/kaggle/input/nba-playoff-seasonal-data/coaches_career.txt
/kaggle/input/nba-playoff-seasonal-data/players.txt
/kaggle/input/nba-playoff-seasonal-data/player_playoffs_career.txt
/kaggle/input/nba-playoff-seasonal-data/readme.txt
/kaggle/input/nba-playoff-seasonal-data/draft.txt
/kaggle/input/nba-playoff-seasonal-data/player_regular_season.txt
/kaggle/input/nba-playoff-seasonal-data/player_playoffs.txt
/kaggle/input/nba-playoff-seasonal-data/player_allstar.txt
/kaggle/input/nba-playoff-seasonal-data/coaches_season.txt
/kaggle/input/nba-playoff-seasonal-data/player_regular_season_career.txt
/kaggle/input/nba-playoff-seasonal-data/teams.txt


In [2]:
import pandasql as psql

# read in csv file into dataframe using pandas
df_team = pd.read_csv("../input/nba-playoff-seasonal-data/team_season.txt",header=0)
df_player_regular = pd.read_csv("../input/nba-playoff-seasonal-data/player_regular_season.txt",header=0) 
df_player_playoff = pd.read_csv("../input/nba-playoff-seasonal-data/player_playoffs.txt",header=0)
df_player_allstar = pd.read_csv("../input/nba-playoff-seasonal-data/player_allstar.txt",header=0)

df_team_info = pd.read_csv("../input/nba-playoff-seasonal-data/teams.txt")

In [3]:
# get required 2002-2004 seasonal data
df_team_02_04 = df_team.loc[(df_team.year >= 2002) & (df_team.year <= 2004)]
df_team_02_04.loc[:,['team', 'year', 'o_pts', 'd_pts', 'won', 'lost']]

,team,year,o_pts,d_pts,won,lost
1099,ATL,2002,7714,8006,35,47
1100,BOS,2002,7599,7631,44,38
1101,CHI,2002,7786,8207,30,52
1102,CLE,2002,7495,8284,17,65
1103,DAL,2002,8444,7806,60,22
...,...,...,...,...,...,...
1182,SAS,2004,7888,7248,59,23
1183,SEA,2004,8113,7925,52,30
1184,TOR,2004,8178,8311,33,49
1185,UTA,2004,7625,7975,26,56


In [4]:
# all star players in year 2002 and 2003
df_allstar = df_player_allstar.loc[(df_player_allstar.year >= 2002) & (df_player_allstar.year <= 2004)]
df_allstar.head(5)

,ilkid,year,firstname,lastname,conference,leag,gp,minutes,pts,dreb,...,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
29,AllenRa02,2003,Ray,Allen,west,N,1,23,16.0,1.0,...,1.0,0.0,3.0,2.0,13.0,6.0,4.0,3.0,4.0,1.0
48,ArtesRo01,2003,Ron,Artest,east,N,1,17,7.0,2.0,...,1.0,0.0,1.0,2.0,5.0,3.0,2.0,1.0,1.0,0.0
172,BryanKo01,2003,Kobe,Bryant,west,N,1,36,20.0,3.0,...,5.0,1.0,6.0,3.0,12.0,9.0,1.0,0.0,3.0,2.0
173,BryanKo01,2002,Kobe,Bryant,west,N,1,36,22.0,5.0,...,3.0,2.0,5.0,5.0,17.0,8.0,6.0,3.0,5.0,3.0
196,CarteVi01,2003,Vince,Carter,east,N,1,16,11.0,1.0,...,2.0,0.0,2.0,1.0,7.0,5.0,0.0,0.0,3.0,1.0


In [5]:
Allstars = psql.sqldf("select DISTINCT ilkid, firstname, lastname from df_allstar group by lastname;")
print(Allstars)

         ilkid firstname    lastname
0   AllenRa02        Ray       Allen
1   ArtesRo01        Ron      Artest
2   BryanKo01       Kobe      Bryant
3   CarteVi01      Vince      Carter
4   CasseSa01        Sam     Cassell
5   DavisBa01      Baron       Davis
6   DuncaTi01        Tim      Duncan
7   FrancSt01      Steve     Francis
8   GarneKe01      Kevin     Garnett
9   IlgauZy01   Zydrunas   Ilgauskas
10  IversAl01      Allen     Iverson
11  JordaMi01    Michael      Jordan
12  KiddJa01       Jason        Kidd
13  KirilAn01     Andrei   Kirilenko
14  MagloJa01     Jamaal    Magloire
15  MarbuSt01    Stephon     Marbury
16  MarioSh01      Shawn      Marion
17  MartiKe01     Kenyon      Martin
18  MashbJa01      Jamal    Mashburn
19  McGraTr01      Tracy     McGrady
20  MilleBr01       Brad      Miller
21  MingYa01         Yao        Ming
22  NashSt01       Steve        Nash
23  NowitDi01       Dirk    Nowitzki
24  ONeaJe01    Jermaine      O'Neal
25  PaytoGa01       Gary      Payton
2

In [6]:
Player_team = psql.sqldf("select ilkid, year, firstname, lastname, team from df_player_playoff where year >= 2002 AND year<= 2004;")
Player_team

,ilkid,year,firstname,lastname,team
0,ABDULTA01,2002,Tariq,Abdul-wahad,DAL
1,ALEXACO02,2002,Courtney,Alexander,NOH
2,AMAECJO01,2002,John,Amaechi,UTA
3,ANDERDE01,2002,Derek,Anderson,POR
4,ANDERKE02,2002,Kenny,Anderson,NOH
...,...,...,...,...,...
558,WESTDE01,2004,Delonte,West,BOS
559,WILKIDA02,2004,Damien,Wilkins,SEA
560,WILLICO02,2004,Corliss,Williamson,SAC
561,WILLIJA02,2004,Jason,Williams,MEM


In [7]:
team_fullname = psql.sqldf("select DISTINCT T.name, T.team from df_team_info as T INNER join Player_team as P on T.team = P.team")
team_fullname

,name,team
0,Celtics,BOS
1,Bulls,CHI
2,Mavericks,DAL
3,Rockets,DEN
4,Nuggets,DEN
5,Pistons,DET
6,Rockets,HOU
7,Pacers,IND
8,Lakers,LAL
9,Grizzlies,MEM


In [8]:
df_allstar_team = psql.sqldf("select distinct P.ilkid, P.firstname, P.lastname, P.team from Player_team as P JOIN Allstars as A ON UPPER(A.ilkid) = UPPER(P.ilkid) and P.lastname = A.lastname;")

NumberOfAllstarPlayer = psql.sqldf("select team, COUNT(team) as NumberOfAllstarPlayer from df_allstar_team group by team")

psql.sqldf("select N.team, T.name, N.NumberOfAllstarPlayer from NumberOfAllstarPlayer as N INNER JOIN team_fullname as T on N.team = T.team")

,team,name,NumberOfAllstarPlayer
0,BOS,Celtics,3
1,DAL,Mavericks,3
2,DEN,Nuggets,1
3,DEN,Rockets,1
4,DET,Pistons,1
5,HOU,Rockets,2
6,IND,Pacers,2
7,LAL,Lakers,2
8,MIL,Bucks,3
9,MIN,Timberwolves,2


In [9]:
regular = psql.sqldf("select * from df_player_regular where year >= 2002 AND year<= 2004;")
playoff = psql.sqldf("select * from df_player_playoff where year >= 2002 AND year<= 2004;")

# efficiency calculation formula from https://en.wikipedia.org/wiki/Efficiency_(basketball)

regular_efficiency = psql.sqldf("SELECT team, ilkid, firstname, lastname, pts, (pts+oreb+dreb+asts+stl+blk-turnover-fgm-ftm)/gp as efficiency from regular group by ilkid order by efficiency desc")
playoff_efficiency = psql.sqldf("SELECT team, ilkid, firstname, lastname, pts, (pts+oreb+dreb+asts+stl+blk-turnover-fgm-ftm)/gp as efficiency from playoff group by ilkid order by efficiency desc")

In [10]:
regular_efficiency

,team,ilkid,firstname,lastname,pts,efficiency
0,MIN,GARNEKE01,Kevin,Garnett,1883,28.939024
1,SAS,DUNCATI01,Tim,Duncan,1884,26.209877
2,SAC,WEBBECH01,Chris,Webber,1542,25.567164
3,ORL,MCGRATR01,Tracy,McGrady,2407,25.186667
4,LAL,ONEASH01,Shaquille,O'Neal,1841,24.537313
...,...,...,...,...,...,...
593,POR,VRANESL01,Slavko,Vranes,0,0.000000
594,DET,REIDDO01,Don,Reid,1,0.000000
595,LAC,POLYNOL01,Olden,Polynice,0,0.000000
596,DAL,PODKOPA01,Pavel,Podkolzine,1,0.000000


In [11]:
playoff_efficiency

,team,ilkid,firstname,lastname,pts,efficiency
0,MIN,GARNEKE01,Kevin,Garnett,162,33
1,SAS,DUNCATI01,Tim,Duncan,593,30
2,LAL,ONEASH01,Shaquille,O'neal,324,29
3,DET,WALLABE01,Ben,Wallace,151,25
4,IND,ONEAJE01,Jermaine,O'neal,137,25
...,...,...,...,...,...,...
339,BOS,BRYANMA01,Mark,Bryant,0,0
340,DEN,BOWENRY01,Ryan,Bowen,2,0
341,BOS,BARRODA01,Dana,Barros,0,0
342,POR,ANDERDE01,Derek,Anderson,2,0


In [12]:
total_efficiency = psql.sqldf("Select P.team, P.ilkid, P.firstname, P.lastname, (P.pts*0.7 + R.pts*0.3) as weighted_pts, (P.efficiency*0.7 + R.efficiency*0.3) as weighted_efficiency from regular_efficiency as R INNER JOIN playoff_efficiency as P on P.firstname=R.firstname and P.lastname=R.lastname order by weighted_efficiency desc limit 100;")

total_efficiency.head(20)

,team,ilkid,firstname,lastname,weighted_pts,weighted_efficiency
0,MIN,GARNEKE01,Kevin,Garnett,678.3,31.781707
1,SAS,DUNCATI01,Tim,Duncan,980.3,28.862963
2,DET,WALLABE01,Ben,Wallace,257.5,24.498630
3,DAL,NOWITDI01,Dirk,Nowitzki,904.3,23.258750
4,PHO,MARIOSH01,Shawn,Marion,592.5,23.125926
5,BOS,PIERCPA01,Paul,Pierce,804.1,22.255696
6,PHI,IVERSAL01,Allen,Iverson,944.6,22.092683
7,NJN,KIDDJA01,Jason,Kidd,729.9,22.015000
8,LAL,BRYANKO01,Kobe,Bryant,1007.8,22.006098
9,SAC,WEBBECH01,Chris,Webber,578.8,21.670149


In [13]:
pts = psql.sqldf("select weighted_pts, weighted_efficiency from total_efficiency;")

In [14]:
# normalization of value

from sklearn import preprocessing

x = pts.values #returns a numpy array 

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

,0,1
0,0.636153,1.000000
1,0.969633,0.863188
2,0.171489,0.658616
3,0.885711,0.600498
4,0.541409,0.594272
...,...,...
95,0.291299,0.038404
96,0.221621,0.034865
97,0.039863,0.019462
98,0.137699,0.008652
